In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

import datetime
import shutil

print("TensorFlow version:", tf.__version__)

# Load the TensorBoard notebook extension
%load_ext tensorboard


TensorFlow version: 2.16.1
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [8]:
# Гиперпараметры
input_dim = 8  # Размерность входных данных
output_dim = 1  # Размерность выходных данных
hidden_layers = [32, 64, 32]  # Количество нейронов в каждом скрытом слое
activation = 'relu'  # Функция активации для скрытых слоев
learning_rate = 0.001
epochs = 30

BATCH_SIZE = 64
BUFFER_SIZE = 128

In [9]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs') / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = Path('./data/models/nn')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [10]:
model = Sequential(name="ModelForTGroup")
model.add(Input(shape=(input_dim, ), name=f"input_layer"))
for i, units in enumerate(hidden_layers, 1):
    model.add(Dense(units, activation=activation, name=f"hidden_layer_{i}"))
model.add(Dense(output_dim, name="output_layer"))

In [11]:
model.summary()

Model: "ModelForTGroup"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)               │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_2 (Dense)               │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer_3 (Dense)               │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,513 (17.63 KB)

 Trainable params: 4,513 (17.63 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Компиляция модели
model.compile(optimizer=Adam(learning_rate=learning_rate), metrics=['mae', 'mse'], loss='mse')

In [13]:
# Подготовка логгирования в TensorBoard
tensorboard_callback = TensorBoard(log_dir=logs_path, histogram_freq=1)
checkpoint_callback = ModelCheckpoint(filepath=model_path / f'{model.name}.keras', save_weights_only=False, save_best_only=True)

In [14]:
# Обучение модели
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=epochs, validation_split=0.2, callbacks=[tensorboard_callback, checkpoint_callback])

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 289ms/step - loss: 136.3019 - mae: 11.6452 - mse: 136.3019 - val_loss: 122.0745 - val_mae: 11.0246 - val_mse: 122.0745
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 115.9494 - mae: 10.7454 - mse: 115.9494 - val_loss: 104.3535 - val_mae: 10.1922 - val_mse: 104.3535
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 98.5392 - mae: 9.9022 - mse: 98.5392 - val_loss: 85.7718 - val_mae: 9.2380 - val_mse: 85.7718
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 79.0243 - mae: 8.8606 - mse: 79.0243 - val_loss: 65.9492 - val_mae: 8.0936 - val_mse: 65.9492
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - loss: 58.8213 - mae: 7.6326 - mse: 58.8213 - val_loss: 44.9396 - val_mae: 6.6621 - val_mse: 44.9396
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - loss: 37.7918 - mae: 6.0821 - mse: 37.7918 - val_loss: 24.3316 - val_mae: 4.8442 - val_mse: 24.3316
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - loss: 17.4287 - mae: 4.0385

In [15]:
%tensorboard --logdir data/logs

Reusing TensorBoard on port 6007 (pid 3968), started 0:58:22 ago. (Use '!kill 3968' to kill it.)

In [15]:
str(logs_path.parent.absolute())

'C:\\Users\\ru-lo\\PycharmProjects\\AICourseTGroup\\LO\\data\\logs'